<a href="https://colab.research.google.com/github/ggoddll99/ds_study/blob/main/240913_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


배치 경사 하강법, 확률적 경사 하강법, 미니배치 경사 하강법

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

학습률이 너무 높다. 학습률을 낮춰야 한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

과소적합의 문제이므로 높은 편향이 문제이다.
높은 편향 : 일반화 오차 중에서 편향은 잘못된 가정으로 인한 것이다. 예를 들어 데이터가 실제로는 2차인데 선형으로 가정하는 경우이다. 편향이 큰 모델은 훈련 데이터에 과소적합되기 쉽다. 규제 하이퍼파라미터 $\alpha$는 줄여야 한다. ($\alpha$를 증가시킬수록 모델의 분산은 줄지만 편향은 커지므로)

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

* 규제가 있는 모델이 없는 모델보다 성능이 좋다.
* 라쏘 모델은 덜 중요한 특성의 가중치를 0으로 만들기 때문에 모델을 단순화할 수 있다.
* 특성 수가 훈련 샘플 수보다 많거나 특성 몇 개가 강하게 연관되어 있을 때는 보통 라쏘가 문제를 일으키므로

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [1]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
y = iris.target

In [2]:
import numpy as np

# 2. 데이터 분할 설정
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

# 3. 데이터를 섞어서 훈련, 검증, 테스트 세트로 나눔
rnd_indices = np.random.permutation(total_size)

X_train = X[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]

X_valid = X[rnd_indices[train_size:train_size + validation_size]]
y_valid = y[rnd_indices[train_size:train_size + validation_size]]

X_test = X[rnd_indices[train_size + validation_size:]]
y_test = y[rnd_indices[train_size + validation_size:]]

# 4. 원-핫 인코딩 함수 정의
def to_one_hot(y):
    n_classes = y.max() + 1  # 클래스의 총 개수를 계산
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))

    # 각 데이터에 맞는 클래스 위치에 1을 할당
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

# 5. 타겟 레이블을 원-핫 인코딩으로 변환
y_train_one_hot = to_one_hot(y_train)
y_valid_one_hot = to_one_hot(y_valid)
y_test_one_hot = to_one_hot(y_test)

# 데이터의 크기 확인 (선택적 출력)
print("훈련 데이터 크기:", X_train.shape, y_train_one_hot.shape)
print("검증 데이터 크기:", X_valid.shape, y_valid_one_hot.shape)
print("테스트 데이터 크기:", X_test.shape, y_test_one_hot.shape)

훈련 데이터 크기: (90, 4) (90, 3)
검증 데이터 크기: (30, 4) (30, 3)
테스트 데이터 크기: (30, 4) (30, 3)


In [3]:
# 3. 소프트맥스 함수
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # 수치 안정성을 위한 최대값 뺄셈
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

# 4. 크로스 엔트로피 손실 함수
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))  # 작은 값을 더해 수치적으로 안정성 확보

    # 5. 소프트맥스 회귀 학습 함수 (경사 하강법)
def train_softmax_regression(X_train, y_train, X_val, y_val, learning_rate=0.01, epochs=5000, tolerance=1e-4, patience=10):
    n_samples, n_features = X_train.shape  # 훈련 데이터의 샘플 수와 특징 수
    n_classes = y_train.shape[1]  # 클래스 수 (원-핫 인코딩된 y_train의 두 번째 차원)

    # 가중치 및 편향 초기화
    weights = np.random.randn(n_features, n_classes)
    biases = np.zeros((1, n_classes))

    best_val_loss = np.inf  # 검증 손실 최솟값 초기화
    patience_counter = 0  # 조기 종료를 위한 patience 카운터

    for epoch in range(epochs):
        # 1. 로짓 계산: WX + b
        logits = np.dot(X_train, weights) + biases
        y_pred = softmax(logits)

        # 2. 손실 함수 계산
        loss = cross_entropy_loss(y_train, y_pred)

        # 3. 경사 계산
        grad_logits = (y_pred - y_train) / n_samples  # y_pred와 y_train의 차이를 평균으로 나눔
        grad_weights = np.dot(X_train.T, grad_logits)  # 가중치에 대한 경사 계산
        grad_biases = np.sum(grad_logits, axis=0, keepdims=True)  # 편향에 대한 경사 계산

        # 4. 가중치 및 편향 업데이트
        weights -= learning_rate * grad_weights
        biases -= learning_rate * grad_biases

        # 5. 검증 데이터에 대한 손실 계산
        val_logits = np.dot(X_val, weights) + biases
        y_val_pred = softmax(val_logits)
        val_loss = cross_entropy_loss(y_val, y_val_pred)

        # 6. 조기 종료 조건 확인
        if val_loss < best_val_loss - tolerance:
            best_val_loss = val_loss
            patience_counter = 0  # 검증 손실이 개선되면 patience_counter 초기화
        else:
            patience_counter += 1  # 개선되지 않으면 patience_counter 증가

        if patience_counter >= patience:
            print(f"조기 종료: {epoch} 에포크에서 종료됨. 검증 손실: {val_loss:.4f}")
            break

        # 7. 매 100 에포크마다 학습 상태 출력
        if epoch % 100 == 0:
            print(f"에포크 {epoch}: 훈련 손실 = {loss:.4f}, 검증 손실 = {val_loss:.4f}")

    return weights, biases

# 6. 모델 학습
weights, biases = train_softmax_regression(X_train, y_train_one_hot, X_valid, y_valid_one_hot)

# 결과 출력
print("훈련 완료 후 가중치:", weights)
print("훈련 완료 후 편향:", biases)

에포크 0: 훈련 손실 = 3.2849, 검증 손실 = 3.2774
에포크 100: 훈련 손실 = 0.7125, 검증 손실 = 0.8067
에포크 200: 훈련 손실 = 0.6141, 검증 손실 = 0.7132
에포크 300: 훈련 손실 = 0.5520, 검증 손실 = 0.6528
에포크 400: 훈련 손실 = 0.5075, 검증 손실 = 0.6086
에포크 500: 훈련 손실 = 0.4729, 검증 손실 = 0.5736
에포크 600: 훈련 손실 = 0.4445, 검증 손실 = 0.5444
에포크 700: 훈련 손실 = 0.4203, 검증 손실 = 0.5192
에포크 800: 훈련 손실 = 0.3993, 검증 손실 = 0.4970
에포크 900: 훈련 손실 = 0.3807, 검증 손실 = 0.4771
에포크 1000: 훈련 손실 = 0.3640, 검증 손실 = 0.4592
에포크 1100: 훈련 손실 = 0.3490, 검증 손실 = 0.4428
에포크 1200: 훈련 손실 = 0.3353, 검증 손실 = 0.4279
에포크 1300: 훈련 손실 = 0.3228, 검증 손실 = 0.4141
에포크 1400: 훈련 손실 = 0.3114, 검증 손실 = 0.4014
에포크 1500: 훈련 손실 = 0.3008, 검증 손실 = 0.3897
에포크 1600: 훈련 손실 = 0.2911, 검증 손실 = 0.3787
에포크 1700: 훈련 손실 = 0.2821, 검증 손실 = 0.3686
에포크 1800: 훈련 손실 = 0.2737, 검증 손실 = 0.3591
에포크 1900: 훈련 손실 = 0.2659, 검증 손실 = 0.3503
에포크 2000: 훈련 손실 = 0.2586, 검증 손실 = 0.3420
에포크 2100: 훈련 손실 = 0.2518, 검증 손실 = 0.3342
에포크 2200: 훈련 손실 = 0.2454, 검증 손실 = 0.3269
에포크 2300: 훈련 손실 = 0.2395, 검증 손실 = 0.3200
에포크 2400: 훈련 손실 = 0.2338, 검증

In [4]:
# 7. 예측 함수
def predict(X, weights, biases):
    logits = np.dot(X, weights) + biases
    y_pred = softmax(logits)
    return np.argmax(y_pred, axis=1)  # 가장 높은 확률을 가진 클래스 반환

# 8. 정확도 계산 함수
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# 9. 테스트 세트에서 평가
y_test_pred = predict(X_test, weights, biases)
test_accuracy = accuracy(y_test, y_test_pred)

# 결과 출력
print(f"테스트 세트 정확도: {test_accuracy:.4f}")

테스트 세트 정확도: 0.9667
